In [3]:
!pip install pythainlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 38.8 MB/s eta 0:00:00


In [4]:
import pandas as pd
from sklearn.metrics import classification_report
from pythainlp.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn import svm
import gdown

In [6]:
pos_text = ("C:/Users/Gumpun/Sentiment-Analysis-System-for-Consumer-Products/DataReviewProductThai/Positive.csv")
neg_text = ("C:/Users/Gumpun/Sentiment-Analysis-System-for-Consumer-Products/DataReviewProductThai/Negative.csv")
neu_text = ("C:/Users/Gumpun/Sentiment-Analysis-System-for-Consumer-Products/DataReviewProductThai/Neutrally.csv")

In [7]:
pos_df = pd.read_csv(pos_text, delimiter=',', encoding='utf-8').dropna()
neg_df = pd.read_csv(neg_text, delimiter=',', encoding='utf-8').dropna()
neu_df = pd.read_csv(neu_text, delimiter=',', encoding='utf-8').dropna()

data = pd.concat([pos_df, neg_df, neu_df])
data.sample(100)

,User,Message,Sentiment
457,9***9,ของห่วยแตก ใช้งานไม่ได้เลย,Negative
839,Nitcharin M.,ไม่ตรงปกผ้าบางมาก ตัวเล็กกว่าไซส์จริงมากๆๆๆๆๆๆ...,Negative
1450,0***9,สินค้าตัวเล็กเกินไป,Negative
879,น.ส.ณัฏฐ์หทัย ส.,ดีตรงปก,Positive
77,โย,สินค้าตรงปกตามที่สั่งซื้อไป ดูดี เหมาะสำหรับใช...,Positive
...,...,...,...
163,มั่น ม.,สมราคาคะ,Neutrally
620,เกียรติศักดิ์ พ.,สิ้นตรวปกสมราคาา,Positive
1246,Pupe P.,ได้รับ สิ้นค้า แล้วค่ะ ผิดหวัง มากค่ะ สั่งไซต์...,Negative
1129,เขมจิรา ผ.,เสื้อส่งมาไซด์เล็กมากผิดไซด์สั่งมาตัวเหมือนกัน...,Negative


In [8]:
# สร้างฟังก์ชันสำหรับการลบอักขระที่ซ้ำกันในแต่ละคำเท่านั้น
def remove_duplicate_chars(text):
    if isinstance(text, str):  # ตรวจสอบว่าข้อความไม่ใช่ NaN
        unique_words = []
        words = word_tokenize(text)
        for word in words:
            unique_word = ''
            for char in word:
                if char not in unique_word:
                    unique_word += char
            unique_words.append(unique_word)
        return ' '.join(unique_words)
    else:
        return text  # ส่งค่า NaN กลับหากเป็น NaN

# ใช้ฟังก์ชัน `remove_duplicate_chars` เพื่อ Tokenize และลบคำที่ซ้ำ
data['Message'] = data['Message'].apply(remove_duplicate_chars)
data.sample(10)

,User,Message,Sentiment
889,6***5,กางเ ขาด,Negative
1156,****1,เสื้อ สวย มาก เลย ค่ะ ไว้ มา อุดหน ใหม่ นะค,Negative
1067,0***8,ดีไซน์ สบาย และ สไตล์ หรู วัสดุ หนังไก่ คุณภ...,Positive
352,maple T.,รังดุม ไม่ เจาะ มา,Neutrally
953,อ***.,เนื้อผา ตาม ราค ซื้อ ไป ตัว ได้ สี เดียวกัน หม...,Neutrally
875,ส***.,สิน ค้าส่ง ตามกำหนด เวลา ส่วน ขนาดที่ ต้องกา...,Positive
1484,Thanawattana P.,ขนาด โคตร เล็ก ทิ้ง,Negative
839,J***.,วัสดุ ดี นุ่ม ไม่ หนา สมราค ถูก ซื้อ ตอน โร ...,Neutrally
833,นิยะดา ศ.,ห่อ สินค้า มา ไม่ ดี ควร ใส่ กล่อง มา เพราะ ...,Negative
27,แหม่ม พ.,ได้ เสื้อผา มา ไม่ ตรง ปก สั่ง ลาย เสือ ได้ ลา...,Neutrally


In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

Message = data['Message'].values
Sentiment = data['Sentiment'].values

# ใช้สำหรับแปลงข้อความเป็นเวกเตอร์ของจำนวนคำที่ปรากฏในข้อความนั้น ๆ
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(Message).toarray()

# Encoding labels
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(Sentiment)

print("Encoded labels:", encoded_labels)
print("Mapping of encoded labels to original labels:")
for label, original_label in enumerate(encoder.classes_):
    print(f"{label}: {original_label}")

Encoded labels: [2 2 2 ... 1 1 1]
Mapping of encoded labels to original labels:
0: Negative
1: Neutrally
2: Positive


In [10]:
# แบ่งข้อมูลเป็นชุดฝึกและชุดทดสอบ
X_train, X_test, y_train, y_test = train_test_split(X, encoded_labels, stratify=encoded_labels, test_size=0.1, random_state=42)

In [11]:
# Train the SVM model
clf = svm.SVC(kernel='linear')
clf.fit(X_train, y_train)

SVC(kernel='linear')

In [12]:
predictions = clf.predict(X_test)

In [13]:
# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.6666666666666666


In [15]:
# Preprocess the input text
test_text_processed = remove_duplicate_chars("การทำงานของร้านค้าช้ามาก กว่าจะส่งของช้ามากๆ")

# Transform the preprocessed text into TF-IDF numerical features
test_text_tfidf = vectorizer.transform([test_text_processed]).toarray()

# Predict the sentiment label using the trained SVM classifier
predicted_label = clf.predict(test_text_tfidf)

# Mapping of predicted label to sentiment
sentiment_mapping = {0: "Negative", 1: "Neutral", 2: "Positive"}

# Print the classification result
print("The text is classified as:", sentiment_mapping[predicted_label[0]])


The text is classified as: Negative
